In [1]:
import pandas as pd
import numpy as np
import flask
import json as js
#import pyproj as pp

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///Hurricane.sqlite', echo=False)
con = engine.connect()

In [3]:
con.execute('''CREATE TABLE IF NOT EXISTS Hurricanes(
                ID integer not null primary key, 
                Name Varchar (100),
                Time integer not null,
                Event Varchar (100),
                Status Varchar (100),
                Latitude integer not null,
                Longitude integer not null,
                Wind integer not null,
                Pressure integer not null,
                ISODate Varchar (100),
                Location Varchar (100)
                )''')


In [4]:
df = pd.read_csv('hurdat.csv', index_col=0)
df.to_sql('Hurricanes', engine, if_exists='replace', index=True, index_label="id")

In [5]:
data = pd.read_sql('Select * from Hurricanes',engine)


In [6]:
data.head()

,id,Name,Date,Time,Event,Status,Latitude,Longitude,Wind,Pressure,ISODate,Location
0,EP011949,UNNAMED,19490611,0,,TS,20.2,-106.3,45,-999,1949-06-11T00:00:00,East Pacific
1,EP011949,UNNAMED,19490611,600,,TS,20.2,-106.4,45,-999,1949-06-11T06:00:00,East Pacific
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,-106.7,45,-999,1949-06-11T12:00:00,East Pacific
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,-107.7,45,-999,1949-06-11T18:00:00,East Pacific
4,EP011949,UNNAMED,19490612,0,,TS,20.4,-108.6,45,-999,1949-06-12T00:00:00,East Pacific


In [7]:
### function takes an ID parameter, gets data from database, converts to and returns geoJson 
## requires geojson to be installed ( pip install geojson )
## requires imports 
from geojson import LineString, Feature, Point, GeometryCollection, FeatureCollection
def makeGeo(id):
    edf=  pd.read_sql('Select * from Hurricanes where id="'+id+'" Order by Date,Time',engine)
    ls=LineString(edf[['Longitude','Latitude']].values.tolist())
    dates=edf['ISODate'].values.tolist()
    winds=edf['Wind'].values.tolist()
    name= edf['Name'].unique()[0].strip() 
    f=Feature(geometry=ls,properties={"id":"AL041966", "name":name , "dates":dates, "winds":winds})
    fc=FeatureCollection([f])
    return fc


In [8]:
fc=makeGeo('AL041966')
fc

{"features": [{"geometry": {"coordinates": [[-41.0, 31.0], [-41.5, 31.5], [-41.7, 31.7], [-41.9, 31.8], [-42.0, 32.0], [-42.0, 32.2], [-42.1, 32.4], [-42.2, 32.7], [-42.1, 33.0], [-41.8, 33.3], [-41.1, 33.8], [-40.5, 34.5], [-40.5, 35.3], [-40.8, 35.9], [-41.4, 36.1], [-41.9, 36.2], [-42.2, 36.4], [-42.2, 36.6], [-41.9, 37.4], [-41.4, 38.3], [-40.8, 38.7], [-40.1, 39.2], [-39.2, 40.1], [-38.5, 40.9], [-38.2, 41.6], [-38.0, 42.4], [-37.7, 43.1], [-37.4, 43.9], [-37.4, 44.6], [-37.4, 45.3], [-37.3, 46.0], [-37.1, 46.8], [-36.4, 47.7], [-36.0, 49.0], [-36.5, 50.5], [-37.0, 52.0], [-38.5, 53.5]], "type": "LineString"}, "properties": {"dates": ["1966-07-22T18:00:00", "1966-07-23T00:00:00", "1966-07-23T06:00:00", "1966-07-23T12:00:00", "1966-07-23T18:00:00", "1966-07-24T00:00:00", "1966-07-24T06:00:00", "1966-07-24T12:00:00", "1966-07-24T18:00:00", "1966-07-25T00:00:00", "1966-07-25T06:00:00", "1966-07-25T12:00:00", "1966-07-25T18:00:00", "1966-07-26T00:00:00", "1966-07-26T06:00:00", "1966-0

In [9]:
def getStyle(windsp):
    if windsp > 156 : return "cat5"
    elif windsp > 129 : return "cat4"
    elif windsp > 110 : return "cat3"
    elif windsp > 95 : return "cat2"
    elif windsp > 73 : return "cat1"
    elif windsp > 38 : return "tstorm"
    else: return "storm"

def getEvents()
    evdf= pd.read_sql('SELECT id, name, min(Date) AS sdate, max(Date) AS edate, max(Wind) AS ws FROM Hurricanes GROUP BY id',engine)
    hurrs=[]
    for index, row in evdf.iterrows():
        st=getStyle(row['ws'])
        sdstr=str(row['sdate'])
        sdate=sdstr[0:4]+"-"+sdstr[4:6]+"-"+sdstr[6:8]
        edstr=str(row['edate'])
        edate=edstr[0:4]+"-"+edstr[4:6]+"-"+edstr[6:8]
        thisevent={ 'id': row['id'], 'start':sdate, 'end':edate, 'content':row['Name'].strip(), 'className':st }
        hurrs.append(thisevent)

hurrs

[{'id': 'AL011851',
  'start': '1851-06-25',
  'end': '1851-06-28',
  'content': 'UNNAMED',
  'className': 'cat1'},
 {'id': 'AL011852',
  'start': '1852-08-19',
  'end': '1852-08-30',
  'content': 'UNNAMED',
  'className': 'cat2'},
 {'id': 'AL011853',
  'start': '1853-08-05',
  'end': '1853-08-05',
  'content': 'UNNAMED',
  'className': 'tstorm'},
 {'id': 'AL011854',
  'start': '1854-06-25',
  'end': '1854-06-27',
  'content': 'UNNAMED',
  'className': 'tstorm'},
 {'id': 'AL011855',
  'start': '1855-08-06',
  'end': '1855-08-06',
  'content': 'UNNAMED',
  'className': 'cat1'},
 {'id': 'AL011856',
  'start': '1856-08-09',
  'end': '1856-08-12',
  'content': 'UNNAMED',
  'className': 'cat4'},
 {'id': 'AL011857',
  'start': '1857-06-30',
  'end': '1857-07-01',
  'content': 'UNNAMED',
  'className': 'tstorm'},
 {'id': 'AL011858',
  'start': '1858-06-12',
  'end': '1858-06-12',
  'content': 'UNNAMED',
  'className': 'tstorm'},
 {'id': 'AL011859',
  'start': '1859-07-01',
  'end': '1859-07-0